In [2]:
import sys
sys.path.append("../")  # path contains python_file.py

In [3]:
import iris

import numpy as np
import scipy.stats

In [4]:
%load_ext autoreload
%autoreload 2
import PACKAGE_NAME.utils

# Testing data my own

In [1]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[0]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

NameError: name 'iris' is not defined

In [5]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [6]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [7]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [8]:
obs = obs[obs > 0]  # Two obs-outliers. Need to check for that.

# Testing data ISIMIP

In [9]:
pr_obs = np.array(iris.load_cube("testing_data/ISIMIP/pr_obs-hist_coarse_1979-2014.nc").data)
pr_cm_hist = np.array(iris.load_cube("testing_data/ISIMIP/pr_sim-hist_coarse_1979-2014.nc").data)
pr_cm_future = np.array(iris.load_cube("testing_data/ISIMIP/pr_sim-fut_coarse_2065-2100.nc").data)

In [57]:
pr_obs = np.moveaxis(pr_obs, -1, 0)
pr_cm_hist = np.moveaxis(pr_cm_hist, -1, 0)
pr_cm_future = np.moveaxis(pr_cm_future, -1, 0)

In [58]:
pr_obs = pr_obs[:, 0, 0]
pr_cm_hist = pr_cm_hist[:, 0, 0]
pr_cm_future = pr_cm_future[:, 0, 0]

# My stuff

In [3]:
from PACKAGE_NAME.debias import ISIMIP

temp_debiaser = ISIMIP(variable="tas")
pr_debiaser = ISIMIP(variable = "pr")

In [60]:
pr_adjusted = temp_debiaser.apply_location(pr_obs, pr_cm_hist, pr_cm_future)

TypeError: only integer scalar arrays can be converted to a scalar index

In [80]:
# Comparison of my code with ISIMIP

## 1. Look at pseudo future observations

In ISIMIP this is generated by `uf.map_quantiles_non_parametric_trend_preserving` with `adjust_obs = True`. By my code by step5

In [4]:
import PACKAGE_NAME.code_isimip.utility_functions as uf

adjusted_obs = temp_debiaser.step5(obs, cm_hist, cm_future)
adjusted_obs_isimip_hopefully = uf.map_quantiles_non_parametric_trend_preserving(
            obs, cm_hist, cm_future, "additive", 431,
            100., 9.,
            True, None, None)

NameError: name 'obs' is not defined

In [82]:
np.max(np.abs(adjusted_obs-adjusted_obs_isimip_hopefully))

0.6362448962064491

In [83]:
from math_helpers import IECDF
from statsmodels.distributions.empirical_distribution import ECDF

# ISIMIP
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs, p_zeroone)
q_sim_hist = uf.percentile1d(cm_hist, p_zeroone)
q_sim_fut = uf.percentile1d(cm_future, p_zeroone)

p = np.interp(obs, q_obs_hist, p_zeroone)

F_sim_fut_inv  = np.interp(p, p_zeroone, q_sim_fut)
F_sim_hist_inv = np.interp(p, p_zeroone, q_sim_hist)
F_obs_hist_inv = np.interp(p, p_zeroone, q_obs_hist)

In [84]:
# My stuff:

# Compute p = F_obs_hist(x) with x in obs_hist
ecdf_obs_hist = ECDF(obs)
p = ecdf_obs_hist(obs)

# Compute iecdfs
iecdf_cm_hist = IECDF(cm_hist)
iecdf_cm_future = IECDF(cm_future)

# Compute q-vals
q_obs_hist = obs  # TODO: = iecdf_obs_hist(p), appears in eq. 7
q_cm_future = iecdf_cm_future(p)
q_cm_hist = iecdf_cm_hist(p)

In [85]:
# Comparison:
np.abs(np.max(q_sim_hist-np.quantile(np.array(cm_hist), p_zeroone)))

0.0

Learning: ISIMIP other than written does not use the empirical inverse cdf, but uses linear interpolation. In addition ISIMIP only uses 432 quantiles (the ones of the smallest array) that are then interpolated up to 518 of the observations. Question is: do we start from the grid or the observations for the ecdf.

In [86]:
ecdf_obs = ECDF(obs, side = "left")
obs_sorted = np.sort(obs)

# ISIMIP
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs_sorted, p_zeroone)
q_sim_hist = uf.percentile1d(cm_hist, p_zeroone)
q_sim_fut = uf.percentile1d(cm_future, p_zeroone)

p = np.interp(obs_sorted, q_obs_hist, p_zeroone)

F_sim_fut_inv  = np.interp(p, p_zeroone, q_sim_fut)
F_sim_hist_inv = np.interp(p, p_zeroone, q_sim_hist)
F_obs_hist_inv = np.interp(p, p_zeroone, q_obs_hist)

ecdf_obs(obs_sorted)-p

#F_obs_hist_inv-np.quantile(np.array(obs), ecdf_obs(obs))

array([ 0.00000000e+00,  5.14823401e-05, -5.38294645e-04,  8.63237998e-04,
       -1.01117235e-03, -2.15622912e-04, -2.34727714e-05,  8.04180659e-04,
        4.46804394e-06, -1.14022554e-03,  7.27352984e-04, -5.66432013e-04,
       -6.20426937e-05, -4.25516667e-04, -4.25516667e-04, -1.07993551e-03,
        1.66068594e-05,  6.30361299e-05, -6.03482004e-05, -3.66194216e-04,
        9.27886997e-04, -7.84149720e-05, -1.47420344e-03,  2.29045966e-04,
       -9.09888770e-05,  3.20183809e-04, -8.32643721e-05,  4.72102255e-04,
       -9.63384122e-04,  1.42679402e-04, -1.06805684e-04, -3.09789583e-04,
       -1.57512933e-04,  2.18688421e-04, -7.27800777e-04, -1.28423319e-03,
       -1.08453560e-04, -3.03714504e-04, -3.44712056e-04, -7.53418380e-04,
        2.43860535e-04, -6.54075482e-05, -1.32311359e-04, -4.54744743e-04,
        2.28333565e-05, -1.00989938e-03,  4.94954986e-04, -1.40023159e-03,
       -1.57071001e-04,  2.93046030e-04, -8.38472093e-04,  8.13218957e-04,
        2.33897908e-04, -

ecdf_obs(obs) should be in theory the same as p. However this is not the case. Partly because of numerical issues: ECDF operates on the line-segment $[a, b)$ so that the highest value is not one. This is different than p_zeroone. However also because of the interpolation. This however is a problem because ecdf with interpolation does not converge always: https://stats.stackexchange.com/questions/230458/why-does-ecdf-uses-a-step-function-and-not-a-linear-interpolation

In [87]:
# ISIMIP
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs, p_zeroone)
q_sim_hist = uf.percentile1d(cm_hist, p_zeroone)
q_sim_fut = uf.percentile1d(cm_future, p_zeroone)

p = np.interp(obs, q_obs_hist, p_zeroone)


# My ISIMIP interpretation:
def isimip_ecdf(x, n_gridpoints):
    p_grid = np.linspace(0., 1., n_gridpoints)
    q_vals_for_p_grid = np.quantile(x, p_grid)
    return np.interp(x, q_vals_for_p_grid, p_grid)

np.max(np.abs(isimip_ecdf(obs, n)-p))

2.7144952952085077e-14

The linear interpolation can be used, but is problematic statistically. Not even if n_gridpoints = x.size it is equivalent to the step-function.
I could use the isimip_ecdf in my code in which case it would be:

In [88]:
# ISIMIP
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs, p_zeroone)
q_sim_hist = uf.percentile1d(cm_hist, p_zeroone)
q_sim_fut = uf.percentile1d(cm_future, p_zeroone)

p = np.interp(obs, q_obs_hist, p_zeroone)

F_sim_fut_inv  = np.interp(p, p_zeroone, q_sim_fut)
F_sim_hist_inv = np.interp(p, p_zeroone, q_sim_hist)
F_obs_hist_inv = np.interp(p, p_zeroone, q_obs_hist)

In [89]:
# My stuff:

# Compute p = F_obs_hist(x) with x in obs_hist
p = isimip_ecdf(obs, 432)

# Compute q-vals
q_obs_hist = obs  # TODO: = iecdf_obs_hist(p), appears in eq. 7
q_cm_future = np.quantile(cm_future, p)
q_cm_hist = np.quantile(cm_future, p)

np.max(np.abs(F_sim_fut_inv-q_cm_future))

/home/jakobwes/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:4650: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


6.252776074688882e-13

ISIMIP and my results are then equivalent if I use the isimip_ecdf and linear interpolation for the iecdf (at least for intermediate steps).

Issues in ISIMIP-code:

1. Inverse empirical cdf of ecdf-value of obs should be obs, but is defined extra in ISIMIP: 

```
q_obs_hist = uf.percentile1d(obs, p_zeroone)
p = np.interp(obs, q_obs_hist, p_zeroone)
F_obs_hist_inv = np.interp(p, p_zeroone, q_obs_hist)
```

If we are adjusting observations this is mathematically the same.

2. ecdf is calculated using linear interpolation. This is mathematically problematic and leads to inconsistent results eg. with ECDF-function from statsmodels.

3. inverse ecdf also includes linear interpolation again using percentile1d-function which is equivalent (up to floating point accuracy) to np.quantile. This includes again by default linear interpolation (as does np.quantile), which is not how the iecdf is commonly defined (as inf of value giving cdf = q).

Other things:

- Other steps in there seem fine and are similar to my code. However there are two additional options with max_change_factor and max_adjustement_factor. Need to see if everything is correct if I use the isimip_ecdf and still check out the other steps (write tests!)
- Usage of assertionerror for wrong trend_preservation is weird
- Checks for lower and upper bound in bounded trend-preservation.
- ccs_transfer_sim2obs for bounded trend preservation: need to check that. 


## 2. Step 6

Notes:

```
i_obs_hist = np.ones(x_obs_hist.shape, dtype=bool)
i_sim_hist = np.ones(x_sim_hist.shape, dtype=bool)
i_sim_fut = np.ones(x_sim_fut.shape, dtype=bool)
if lower:
        i_obs_hist = np.logical_and(i_obs_hist, x_obs_hist > lower_threshold)
        i_sim_hist = np.logical_and(i_sim_hist, x_sim_hist > lower_threshold)
        i_sim_fut = np.logical_and(i_sim_fut, x_sim_fut > lower_threshold)
    if upper:
        i_obs_hist = np.logical_and(i_obs_hist, x_obs_hist < upper_threshold)
        i_sim_hist = np.logical_and(i_sim_hist, x_sim_hist < upper_threshold)
        i_sim_fut = np.logical_and(i_sim_fut, x_sim_fut < upper_threshold)
```

is really elegant for filtering of values between thresholds and not recomputing indices

- ISIMIP has the option `unconditional_ccs_transfer = False` in which case step5 (pseudo future observations) is applied only to values between thresholds and those are used as bounds. This is not documented in the ISIMIP-paper. However the default is `= False` which is inconsistent with refererence and will probably show in results (l. 126 in bias_adjustment.py). For comparison need to ensure `= True`

- Do one thing does not seem to be a thing for ISIMIP-development. Calculation of indices of values which are set to a lower and upper bound goes via `ccs_transfer_sim2obs` a function that normally accepts arrays, does smth. else but also accepts scalars, which are converted to arrays to do some multiplication. Those indices are calculated and set in two different if-clauses. Calculation l. 143-159, setting l. 165 - 189

- l. 244 `# do non-parametric quantile mapping if fitting failed` using Cannon et al-method and map_quantiles_non_parametric from above. Do we want to support that?

- p_value_eps and p-value threhsolding (away from zero and under 1). Not named in paper, comes from Switanek. Do we want to include that?

- lots of wrappers for fit of distribution. What error-capturing is good?

Main transfer happens below:

```python
# compute source p-values
limit_p_values = lambda p : np.maximum(p_value_eps,
                            np.minimum(1-p_value_eps, p))
p_source = limit_p_values(spsdotwhat.cdf(
           x_source_map, *shape_loc_scale_source))

# compute target p-values
if adjust_p_values:
    x_obs_hist_fit = x_obs_hist[i_obs_hist]
    x_sim_hist_fit = x_sim_hist[i_sim_hist]
    shape_loc_scale_obs_hist = uf.fit(spsdotwhat,
                               x_obs_hist_fit, fwords)
    shape_loc_scale_sim_hist = uf.fit(spsdotwhat,
                               x_sim_hist_fit, fwords)
    if shape_loc_scale_obs_hist is None \
    or shape_loc_scale_sim_hist is None:
        msg = 'unable to adjust p-values: leaving them unadjusted'
        warnings.warn(msg)
        p_target = p_source
    else:
        p_obs_hist = limit_p_values(spsdotwhat.cdf(
                     x_obs_hist_fit, *shape_loc_scale_obs_hist))
        p_sim_hist = limit_p_values(spsdotwhat.cdf(
                     x_sim_hist_fit, *shape_loc_scale_sim_hist))
        p_target = limit_p_values(uf.transfer_odds_ratio(
                   p_obs_hist, p_sim_hist, p_source))
else:
    p_target = p_source

# map quantiles
y[i_source] = spsdotwhat.ppf(p_target, *shape_loc_scale_target)
break
```


- `adjust_p_values` determines whether complicated adjustment using logit-values is happening or not. However default is False. For tas should be False, but True for others.

- what does 
```
            if lower or upper:
                # map the values in x_source to be quantile-mapped such that
                # their empirical distribution matches the empirical
                # distribution of the within-threshold values of x_sim_fut
                x_source_map = uf.map_quantiles_non_parametric_brute_force(
                    x_source[i_source], x_source_fit)
```
do?


In [61]:
from code_isimip.bias_adjustment import adjust_bias_one_month, map_quantiles_parametric_trend_preserving
import code_isimip.utility_functions as uf
from isimip import ISIMIP

adjusted_obs = temp_debiaser.step5(obs, cm_hist, cm_future)
adjusted_obs_isimip_hopefully = uf.map_quantiles_non_parametric_trend_preserving(
            obs, cm_hist, cm_future, "additive", 431,
            100., 9.,
            True, None, None)

temp_debiaser = ISIMIP(variable="tas")
step6_using_isimip_obs_future = temp_debiaser.step6(obs, adjusted_obs_isimip_hopefully, cm_hist, cm_future)
isimip_mapping = map_quantiles_parametric_trend_preserving(obs, cm_hist, cm_future, distribution = "normal", trend_preservation = "additive", adjust_p_values=False)
np.max(np.abs(step6_using_isimip_obs_future-isimip_mapping))

TypeError: only integer scalar arrays can be converted to a scalar index

In [62]:
cm_future_argsort = np.argsort(cm_future)
cm_future_sorted = cm_future[cm_future_argsort]

obs_hist_sorted = np.sort(obs)
obs_future_sorted = np.sort(adjusted_obs_isimip_hopefully)
cm_hist_sorted = np.sort(cm_hist)

In [63]:
x_obs_hist = obs_hist_sorted
x_sim_hist = cm_hist_sorted
x_sim_fut = cm_future_sorted
distribution = "normal"
trend_preservation = 'additive'
adjust_p_values = False
lower_bound=None
lower_threshold=None
upper_bound=None
upper_threshold=None
unconditional_ccs_transfer=False
trendless_bound_frequency=False
n_quantiles=431 #50
p_value_eps=1e-10
max_change_factor=100.
max_adjustment_factor=9.


lower = lower_bound is not None and lower_threshold is not None
upper = upper_bound is not None and upper_threshold is not None

# use augmented quantile delta mapping to transfer the simulated
# climate change signal to the historical observation
i_obs_hist = np.ones(x_obs_hist.shape, dtype=bool)
i_sim_hist = np.ones(x_sim_hist.shape, dtype=bool)
i_sim_fut = np.ones(x_sim_fut.shape, dtype=bool)
if lower:
    i_obs_hist = np.logical_and(i_obs_hist, x_obs_hist > lower_threshold)
    i_sim_hist = np.logical_and(i_sim_hist, x_sim_hist > lower_threshold)
    i_sim_fut = np.logical_and(i_sim_fut, x_sim_fut > lower_threshold)
if upper:
    i_obs_hist = np.logical_and(i_obs_hist, x_obs_hist < upper_threshold)
    i_sim_hist = np.logical_and(i_sim_hist, x_sim_hist < upper_threshold)
    i_sim_fut = np.logical_and(i_sim_fut, x_sim_fut < upper_threshold)
if unconditional_ccs_transfer:
    # use all values
    x_target = uf.map_quantiles_non_parametric_trend_preserving(
        x_obs_hist, x_sim_hist, x_sim_fut,
        trend_preservation, n_quantiles,
        max_change_factor, max_adjustment_factor,
        True, lower_bound, upper_bound)
else:
    # use only values within thresholds
    x_target = x_obs_hist.copy()
    x_target[i_obs_hist] = uf.map_quantiles_non_parametric_trend_preserving(
        x_obs_hist[i_obs_hist], x_sim_hist[i_sim_hist],
        x_sim_fut[i_sim_fut], trend_preservation, n_quantiles,
        max_change_factor, max_adjustment_factor,
        True, lower_threshold, upper_threshold)

# determine extreme value probabilities of future obs
if lower:
    p_lower = lambda x : np.mean(x <= lower_threshold)
    p_lower_target = p_lower(x_obs_hist) \
        if trendless_bound_frequency else uf.ccs_transfer_sim2obs(
        p_lower(x_obs_hist), p_lower(x_sim_hist), p_lower(x_sim_fut))
if upper:
    p_upper = lambda x : np.mean(x >= upper_threshold)
    p_upper_target = p_upper(x_obs_hist) \
        if trendless_bound_frequency else uf.ccs_transfer_sim2obs(
        p_upper(x_obs_hist), p_upper(x_sim_hist), p_upper(x_sim_fut))
if lower and upper:
    p_lower_or_upper_target = p_lower_target + p_upper_target
    if p_lower_or_upper_target > 1 + 1e-10:
        msg = 'sum of p_lower_target and p_upper_target exceeds one'
        warnings.warn(msg)
        p_lower_target /= p_lower_or_upper_target
        p_upper_target /= p_lower_or_upper_target

# do a parametric quantile mapping of the values within thresholds
x_source = x_sim_fut
y = x_source.copy()

# determine indices of values to be mapped
i_source = np.ones(x_source.shape, dtype=bool)
i_target = np.ones(x_target.shape, dtype=bool)
if lower:
    # make sure that lower_threshold_source < x_source 
    # because otherwise sps.beta.ppf does not work
    lower_threshold_source = \
        uf.percentile1d(x_source, np.array([p_lower_target]))[0] \
        if p_lower_target > 0 else lower_bound if not upper else \
        lower_bound - 1e-10 * (upper_bound - lower_bound)
    i_lower = x_source <= lower_threshold_source
    i_source = np.logical_and(i_source, np.logical_not(i_lower))
    i_target = np.logical_and(i_target, x_target > lower_threshold)
    y[i_lower] = lower_bound
if upper:
    # make sure that x_source < upper_threshold_source
    # because otherwise sps.beta.ppf does not work
    upper_threshold_source = \
        uf.percentile1d(x_source, np.array([1.-p_upper_target]))[0] \
        if p_upper_target > 0 else upper_bound if not lower else \
        upper_bound + 1e-10 * (upper_bound - lower_bound)
    i_upper = x_source >= upper_threshold_source
    i_source = np.logical_and(i_source, np.logical_not(i_upper))
    i_target = np.logical_and(i_target, x_target < upper_threshold)
    y[i_upper] = upper_bound

# map quantiles
while np.any(i_source):
    # break here if target distributions cannot be determined
    if not np.any(i_target):
        msg = 'unable to do any quantile mapping' \
            + ': leaving %i value(s) unadjusted'%np.sum(i_source)
        warnings.warn(msg)
        break

    # use the within-threshold values of x_sim_fut for the source
    # distribution fitting
    x_source_fit = x_source[i_sim_fut]
    x_target_fit = x_target[i_target]

    # determine distribution parameters
    spsdotwhat = scipy.stats.norm if distribution == 'normal' else \
                 scipy.stats.weibull_min if distribution == 'weibull' else \
                 scipy.stats.gamma if distribution == 'gamma' else \
                 scipy.stats.beta if distribution == 'beta' else \
                 scipy.stats.rice if distribution == 'rice' else \
                 None
    if spsdotwhat is None:
        # prepare non-parametric quantile mapping
        x_source_map = x_source[i_source]
        shape_loc_scale_source = None
        shape_loc_scale_target = None
    else:
        # prepare parametric quantile mapping
        if lower or upper:
            # map the values in x_source to be quantile-mapped such that
            # their empirical distribution matches the empirical
            # distribution of the within-threshold values of x_sim_fut
            x_source_map = uf.map_quantiles_non_parametric_brute_force(
                x_source[i_source], x_source_fit)
        else:
            x_source_map = x_source

        # fix location and scale parameters for fitting
        floc = lower_threshold if lower else None
        fscale = upper_threshold - lower_threshold \
            if lower and upper else None

        # because sps.rice.fit and sps.weibull_min.fit cannot handle
        # fscale=None
        if distribution in ['rice', 'weibull']:
            fwords = {'floc': floc}
        else:
            fwords = {'floc': floc, 'fscale': fscale}

        # fit distributions to x_source and x_target
        shape_loc_scale_source = uf.fit(spsdotwhat, x_source_fit, fwords)
        shape_loc_scale_target = uf.fit(spsdotwhat, x_target_fit, fwords)

    # do non-parametric quantile mapping if fitting failed
    if shape_loc_scale_source is None or shape_loc_scale_target is None:
        msg = 'unable to do parametric quantile mapping' \
            + ': doing non-parametric quantile mapping instead'
        if spsdotwhat is not None: warnings.warn(msg)
        p_zeroone = np.linspace(0., 1., n_quantiles + 1)
        q_source_fit = uf.percentile1d(x_source_map, p_zeroone)
        q_target_fit = uf.percentile1d(x_target_fit, p_zeroone)
        y[i_source] = \
            uf.map_quantiles_non_parametric_with_constant_extrapolation(
            x_source_map, q_source_fit, q_target_fit)
        break

    # compute source p-values
    limit_p_values = lambda p : np.maximum(p_value_eps,
                                np.minimum(1-p_value_eps, p))
    p_source = limit_p_values(spsdotwhat.cdf(
               x_source_map, *shape_loc_scale_source))

    # compute target p-values
    if adjust_p_values:
        x_obs_hist_fit = x_obs_hist[i_obs_hist]
        x_sim_hist_fit = x_sim_hist[i_sim_hist]
        shape_loc_scale_obs_hist = uf.fit(spsdotwhat,
                                   x_obs_hist_fit, fwords)
        shape_loc_scale_sim_hist = uf.fit(spsdotwhat,
                                   x_sim_hist_fit, fwords)
        if shape_loc_scale_obs_hist is None \
        or shape_loc_scale_sim_hist is None:
            msg = 'unable to adjust p-values: leaving them unadjusted'
            warnings.warn(msg)
            p_target = p_source
        else:
            p_obs_hist = limit_p_values(spsdotwhat.cdf(
                         x_obs_hist_fit, *shape_loc_scale_obs_hist))
            p_sim_hist = limit_p_values(spsdotwhat.cdf(
                         x_sim_hist_fit, *shape_loc_scale_sim_hist))
            p_target = limit_p_values(uf.transfer_odds_ratio(
                       p_obs_hist, p_sim_hist, p_source))
    else:
        p_target = p_source

    # map quantiles
    y[i_source] = spsdotwhat.ppf(p_target, *shape_loc_scale_target)
    break


In [64]:
obs_future_isimip = uf.map_quantiles_non_parametric_trend_preserving(
            obs_hist_sorted, cm_hist_sorted, cm_future_sorted, "additive", 431,
            100., 9.,
            True, None, None)

test = temp_debiaser._step6_values_between_thresholds(obs_hist_sorted, obs_future_isimip, cm_hist_sorted, cm_future_sorted) 
shape_loc_scale_target

(1.4755333043157108, 0.6472315906684176)

In [191]:
np.array(test[2])-np.array(shape_loc_scale_target)

array([0., 0.])

In [200]:
np.any(y[i_source] != y)

False

In [264]:
all(np.isclose(test[0],y))

True

This means:

- given the same input from step5 ISIMIP returns the same as my step6 for tas. Need to make sure however:
    - use np.isclose because of nuerical accuracy of isimip is float32 whilst I am working with float64
    - same number of n_quantiles = 431 = min(cm.size, ...) - 1
    - right configuration.
    - sorting, because I resort afterwards. Can test also with resorting, so full step6
- after break need to implement that as test: 
    - test step5 first as seen above, possibly change with isimip-ecdf (interpolation-option)
    - test step6 with input from step5 from isimip
    - test whole
- switch around options. possibly introduce adjust_p_values for temp-exception, as well as other isimip-options, such ad max_adjust fators, ...

In [258]:
a = spsdotwhat.ppf(p_target, *shape_loc_scale_target)
a.dtype
#y.dtype

dtype('float64')

In [252]:
y[i_source] = spsdotwhat.ppf(p_target, *shape_loc_scale_target)

In [253]:
y[i_source]-spsdotwhat.ppf(p_target, *shape_loc_scale_target)

masked_array(data=[7.097963020896714e-06, 8.353601742783212e-06,
                   9.53452820340317e-06, 9.97920136569519e-06,
                   -1.3030023183091544e-05, -8.664078222864191e-06,
                   -1.3735023742356134e-05, -7.259612232246582e-06,
                   -1.4229404882826202e-05, 1.2333898041561042e-05,
                   -1.1959876701439498e-05, 1.2641828050163895e-05,
                   -2.934598114734399e-06, -9.91298719554834e-06,
                   -5.509592313046596e-06, 8.07920764600567e-06,
                   5.741609356846311e-07, -1.1832811651402153e-05,
                   -3.8152900287968805e-06, -6.137170998954389e-06,
                   -1.3099872376187705e-05, -1.0753707215371833e-05,
                   -1.1573450024116028e-05, 7.452529871443403e-06,
                   1.335007073066663e-05, -1.1122825128495606e-05,
                   -2.9833472012796847e-06, -4.00943673639631e-06,
                   6.238484104414965e-06, 1.1061367104048259e-05

In [155]:
spsdotwhat.ppf(p_target, *shape_loc_scale_target) - temp_debiaser.distribution.ppf(test[1], *(test[2]))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Open questions:

- How much do we use ISIMIP-code and how shall tests look like?
- Do we stay on ISIMIP-code or our own implementation following the algorith,. Shall we use an isimip_mode where we either use the code or eg. iecdf with interpolation?

In [226]:
from math_helpers import IECDF
from statsmodels.distributions.empirical_distribution import ECDF

# ISIMIP
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs, p_zeroone)
q_sim_hist = uf.percentile1d(cm_hist, p_zeroone)
q_sim_fut = uf.percentile1d(cm_future, p_zeroone)

p = np.interp(obs, q_obs_hist, p_zeroone)

F_sim_fut_inv  = np.interp(p, p_zeroone, q_sim_fut)
F_sim_hist_inv = np.interp(p, p_zeroone, q_sim_hist)
F_obs_hist_inv = np.interp(p, p_zeroone, q_obs_hist)

In [224]:
# My stuff:

# Compute p = F_obs_hist(x) with x in obs_hist
ecdf_obs_hist = ECDF(obs)
p = ecdf_obs_hist(obs)

# Compute iecdfs
iecdf_cm_hist = IECDF(cm_hist)
iecdf_cm_future = IECDF(cm_future)

# Compute q-vals
q_obs_hist = obs  # TODO: = iecdf_obs_hist(p), appears in eq. 7
q_cm_future = iecdf_cm_future(p)
q_cm_hist = iecdf_cm_hist(p)

I could use the isimip_ecdf in my code in which case it would be:

In [231]:
F_sim_hist_inv-np.quantile(np.array(cm_hist), np.linspace(0, 1, 518))

array([  2.15385457,   3.67471597,   5.08157353,   6.49478329,
         9.66711636,  15.68944595,  18.10632357,  16.22902317,
        13.65035137,   8.54377092,   4.08402244,   3.46516601,
        -0.15306102,   3.03641286,   2.84963924,   5.2318289 ,
         8.20565053,  12.86147095,  14.41004826,  17.76438181,
        14.00231765,   6.58734519,   1.46093782,   0.31998249,
         0.93566268,  -0.77818707,   5.90502328,   5.52307632,
         7.72435443,  12.02882419,  15.49111856,  17.06438846,
        13.59585689,   6.52180032,   3.51666692,   0.25541172,
         0.48898202,   1.12568989,   1.96375293,   4.32115123,
         8.26294386,  15.52450926,  19.13203301,  15.3649094 ,
        15.06852102,   6.29277559,   3.50740372,   0.55259384,
         1.71947736,  -1.63384107,   2.26320928,   3.40422777,
         6.32001955,  14.28211766,  20.24567854,  17.38803455,
        12.48319911,   6.07470624,   2.12883595,  -0.08215119,
         0.06176286,  -0.76031403,   0.30263949,   4.07

In [228]:
ecdf_obs_hist = ECDF(obs)
q_obs = ecdf_obs_hist(obs)

# ISIMIP:
n_quantiles = 431
n = min([n_quantiles + 1, obs.size, cm_hist.size, cm_future.size])
p_zeroone = np.linspace(0., 1., n)

# compute quantiles of input data
q_obs_hist = uf.percentile1d(obs, p_zeroone)
p = np.interp(obs, q_obs_hist, p_zeroone)

# ISIMIP

In [146]:
from code_isimip.bias_adjustment import adjust_bias_one_month, map_quantiles_parametric_trend_preserving

"""test = adjust_bias_one_month(
    data = {'obs_hist': [obs], 'sim_hist': [cm_hist], 'sim_fut': [cm_future]},
    years = {'obs_hist': list(range(1979, 2023)), 'sim_hist': list(range(1986, 2023)), 'sim_fut': list(range(2023, 2059))},
    long_term_mean = {'obs_hist' : [obs.mean()], 'sim_hist' : [cm_hist.mean()], 'sim_fut' : [cm_future.mean()]},
    distribution = ['norm']
)"""

"test = adjust_bias_one_month(\n    data = {'obs_hist': [obs], 'sim_hist': [cm_hist], 'sim_fut': [cm_future]},\n    years = {'obs_hist': list(range(1979, 2023)), 'sim_hist': list(range(1986, 2023)), 'sim_fut': list(range(2023, 2059))},\n    long_term_mean = {'obs_hist' : [obs.mean()], 'sim_hist' : [cm_hist.mean()], 'sim_fut' : [cm_future.mean()]},\n    distribution = ['norm']\n)"

In [169]:
#test = map_quantiles_parametric_trend_preserving(obs, cm_hist, cm_future, distribution = "normal", trend_preservation = "mixed", adjust_p_values=True)

pr_isimip_adjusted = map_quantiles_parametric_trend_preserving(
    obs, cm_hist, cm_future, 
    distribution = "gamma", 
    trend_preservation = "mixed",
    adjust_p_values = False,
    lower_bound = 0,
    lower_threshold = 0.1 / 86400,
    upper_bound = None,
    upper_threshold = None
)

#test2 = map_quantiles_parametric_trend_preserving(obs, cm_hist, cm_future, distribution = "norm", trend_preservation = "multiplicative")
#test3 = map_quantiles_parametric_trend_preserving(obs, cm_hist, cm_future, distribution = "norm", trend_preservation = "mixed")

In [170]:
np.max(np.abs(pr_isimip_adjusted - pr_adjusted))/np.mean(pr_adjusted)

20.56167962652684

In [ ]:
# difference ist immer im range 0.06 für temp
# adjust_p_values ist die tas-Ausnahme (True heißt die werden adjusted) denke ich. WO kommt das in isimip code rein?